In [6]:
import numpy as np
import matplotlib.pyplot as plt
import tensorflow as tf
import tensorflow.keras as keras
from utils import set_seed
from plot_utils import plot_eval_results
from converters import Converter
from config import cfg
from c_utils import write_model_h
from utils import save_test_data
import plot_utils as pu
from model import save_pruned_model


In [8]:
from nyuv2_torch_ds_adapter import get_tf_nyuv2_ds
import argparse
args = argparse.Namespace()
args.truncate_testset = False
args.crop_size = (640, 480)
args.target_size = cfg.img_size
args.out_fold_ratio = 1
args.is_maxim = False
cfg.do_overfit=False
args.batch_size=cfg.batch_size*4

ds_train, ds_val, ds_test = get_tf_nyuv2_ds(cfg.base_dataset_dir, args)

x_val= next(iter(ds_val))
x_train= next(iter(ds_train))

Dataset: NYU Depth V2
# of train images: 50688
Dataset: NYU Depth V2
# of test images: 654
/home/adam-torek/AIHardware_FinalProject/data/nyu2_train/living_room_0038_out/37.jpg
/home/adam-torek/AIHardware_FinalProject/data/nyu2_train/living_room_0038_out/37.png
Train size:  40551
Val size:  10137
Test size:  654
/home/adam-torek/AIHardware_FinalProject/data/nyu2_train/kitchen_0011a_out/96.jpg
/home/adam-torek/AIHardware_FinalProject/data/nyu2_train/kitchen_0011a_out/96.png
/home/adam-torek/AIHardware_FinalProject/data/nyu2_train/bathroom_0039_out/27.jpg
/home/adam-torek/AIHardware_FinalProject/data/nyu2_train/bathroom_0039_out/27.png


2024-11-06 00:38:59.873350: W tensorflow/core/framework/op_kernel.cc:1829] INVALID_ARGUMENT: TypeError: `generator` yielded an element of shape () where an element of shape (32, 32, 1) was expected.
Traceback (most recent call last):

  File "/home/adam-torek/miniconda3/envs/ai_hardware/lib/python3.10/site-packages/tensorflow/python/ops/script_ops.py", line 269, in __call__
    ret = func(*args)

  File "/home/adam-torek/miniconda3/envs/ai_hardware/lib/python3.10/site-packages/tensorflow/python/autograph/impl/api.py", line 643, in wrapper
    return func(*args, **kwargs)

  File "/home/adam-torek/miniconda3/envs/ai_hardware/lib/python3.10/site-packages/tensorflow/python/data/ops/from_generator_op.py", line 235, in generator_py_func
    raise TypeError(

TypeError: `generator` yielded an element of shape () where an element of shape (32, 32, 1) was expected.




InvalidArgumentError: {{function_node __wrapped__IteratorGetNext_output_types_2_device_/job:localhost/replica:0/task:0/device:CPU:0}} TypeError: `generator` yielded an element of shape () where an element of shape (32, 32, 1) was expected.
Traceback (most recent call last):

  File "/home/adam-torek/miniconda3/envs/ai_hardware/lib/python3.10/site-packages/tensorflow/python/ops/script_ops.py", line 269, in __call__
    ret = func(*args)

  File "/home/adam-torek/miniconda3/envs/ai_hardware/lib/python3.10/site-packages/tensorflow/python/autograph/impl/api.py", line 643, in wrapper
    return func(*args, **kwargs)

  File "/home/adam-torek/miniconda3/envs/ai_hardware/lib/python3.10/site-packages/tensorflow/python/data/ops/from_generator_op.py", line 235, in generator_py_func
    raise TypeError(

TypeError: `generator` yielded an element of shape () where an element of shape (32, 32, 1) was expected.


	 [[{{node PyFunc}}]] [Op:IteratorGetNext] name: 

2024-11-06 00:38:59.934209: W tensorflow/core/framework/op_kernel.cc:1829] INVALID_ARGUMENT: TypeError: `generator` yielded an element of shape () where an element of shape (32, 32, 1) was expected.
Traceback (most recent call last):

  File "/home/adam-torek/miniconda3/envs/ai_hardware/lib/python3.10/site-packages/tensorflow/python/ops/script_ops.py", line 269, in __call__
    ret = func(*args)

  File "/home/adam-torek/miniconda3/envs/ai_hardware/lib/python3.10/site-packages/tensorflow/python/autograph/impl/api.py", line 643, in wrapper
    return func(*args, **kwargs)

  File "/home/adam-torek/miniconda3/envs/ai_hardware/lib/python3.10/site-packages/tensorflow/python/data/ops/from_generator_op.py", line 235, in generator_py_func
    raise TypeError(

TypeError: `generator` yielded an element of shape () where an element of shape (32, 32, 1) was expected.


